In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

%matplotlib inline


In [3]:
from google.colab import files

uploaded = files.upload()

Saving X_seq.csv to X_seq.csv


In [4]:
uploaded2 = files.upload()

Saving Y_seq.csv to Y_seq.csv


In [5]:
#read final cleaned X and Y data sets
X_full=pd.read_csv('X_seq.csv',low_memory=False)
y=pd.read_csv('Y_seq.csv',low_memory=False)

In [6]:
cols_to_drop = [x for x in X_full.columns if 'Unnamed' in x]
X_full = X_full.drop(cols_to_drop, axis=1)
cols_to_drop = [x for x in y.columns if 'Unnamed' in x]
y = y.drop(cols_to_drop, axis=1)

In [7]:
x_train, x_test, y_train, y_test = train_test_split(X_full, y, random_state=0)

In [8]:
#randomly select dataset by M number of labels
def rand_prototypes(M, train_data, train_labels):
    np.random.seed(10)
    indices = np.random.choice( len(train_labels) , M, replace=False)
    return train_data.iloc[:,indices], indices

In [61]:
from sklearn.datasets import make_regression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import classification_report, r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

K neighbors

In [10]:
# define model
model = KNeighborsRegressor()
# fit model
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
# summarize prediction
print(y_pred[0])

[1.79461069e-04 7.20366732e-02 8.96051157e-01]


In [11]:
print('MSE: {}'.format(mean_squared_error(y_test, y_pred)))
print(model.score(x_test, y_test))

MSE: 0.0022796685474204027
0.7627693141391253


In [46]:
params =  {'leaf_size':[1, 5, 15, 30, 40],'n_neighbors':[1,5, 15], 'p':[1,2]}
modelcv = GridSearchCV(KNeighborsRegressor(), params, cv=5)
knr = modelcv.fit(x_train,y_train)


In [47]:
print("best params:", knr.best_params_)
y_pred = knr.predict(x_test)
r2avg=[]
for i in range(len(y_test.columns)):
  r2_i=r2_score(y_test.iloc[:,i], y_pred[:,i])
  print("{} r2 score: {}".format(y_test.columns[i],r2_i))
  r2avg.append(r2_i)
print('Average score: {}'.format(np.average(r2avg)))

best params: {'leaf_size': 1, 'n_neighbors': 1, 'p': 1}
Num Align Hqmt r2 score: 0.8835223899723604
Mode Mt Base Call Read Length Align Hqmt r2 score: 0.6006643014163738
Avg Mt Align Edit Percent Identical r2 score: 0.9508881876552193
Average score: 0.8116916263479844


A decent first model, K Neighbors regression gave some promising results, but struggled to predict read length. Grid search did significantly improve the R2 score, but we found better model options down the line. 

Random Forest Regressor

In [48]:
rfr=RandomForestRegressor()
# fit model
rfr.fit(x_train, y_train)
y_pred = rfr.predict(x_test)
# summarize prediction
print(y_pred[0])

[8.11800899e-04 7.12835625e-02 8.81036524e-01]


In [49]:
print('MSE: {}'.format(mean_squared_error(y_test, y_pred)))
print(rfr.score(x_test, y_test))

MSE: 0.0007341469061157294
0.8933723261605472


In [50]:
#prototype selection using 100 random features
x_proto, idx=rand_prototypes(100, x_train, x_train.columns)

In [56]:
params =  {'n_estimators':[10, 50 , 100, 150], 'min_samples_split':[2 ,5, 10], 'min_samples_leaf':[1,3, 5]}
clf = GridSearchCV(RandomForestRegressor(), params, cv=5)
rfc = clf.fit(x_proto,y_train)


In [58]:
print("best params:", rfc.best_params_)
y_pred = rfc.predict(x_test.iloc[:,idx])
r2avg=[]
for i in range(len(y_test.columns)):
  r2_i=r2_score(y_test.iloc[:,i], y_pred[:,i])
  print("{} r2 score: {}".format(y_test.columns[i],r2_i))
  r2avg.append(r2_i)
print('Average score: {}'.format(np.average(r2avg)))

best params: {'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 150}
Num Align Hqmt r2 score: 0.9689970235095027
Mode Mt Base Call Read Length Align Hqmt r2 score: 0.6932278268192855
Avg Mt Align Edit Percent Identical r2 score: 0.9707822690838479
Average score: 0.8776690398042121


In [59]:
rfr=RandomForestRegressor(n_estimators=150, min_samples_leaf=1, min_samples_split=2)
# fit model
rfr.fit(x_train, y_train)
y_pred = rfr.predict(x_test)
# summarize prediction
print(y_pred[0])
print('MSE: {}'.format(mean_squared_error(y_test, y_pred)))
print(rfr.score(x_test, y_test))

[5.83397836e-04 7.12333552e-02 8.80944660e-01]
MSE: 0.0007333708340249203
0.8933855360845273


Random Forest regression seems to be our best model so far as it has some success predicting read length, which will prove to be the difference maker for most of our models. Notably, it is slower than other options, though not as bad as a neural network model on the full data set. We had to trim down the data for grid search, but were able to use the full data set after finding a set of useable parameters. 

Decision Tree Regressor

In [64]:
# define model
dtr = DecisionTreeRegressor()
# fit model
dtr.fit(x_train, y_train)
# make a prediction
y_pred = dtr.predict(x_test)
# summarize prediction
print(y_pred[0])
print('MSE: {}'.format(mean_squared_error(y_test, y_pred)))
print(dtr.score(x_test, y_test))

[1.55487581e-04 7.20366732e-02 8.80282892e-01]
MSE: 0.0009112785051370908
0.871896309204061


In [67]:
params = {"criterion": ["squared_error", "absolute_error"],
              "min_samples_split": [10, 20, 40],
              "min_samples_leaf": [20, 40, 100],
              "max_leaf_nodes": [5, 20, 100],
              }
clf = GridSearchCV(DecisionTreeRegressor(), params, cv=5)
dtr = clf.fit(x_train,y_train)


In [68]:
print("best params:", dtr.best_params_)
y_pred = dtr.predict(x_test)
r2avg=[]
for i in range(len(y_test.columns)):
  r2_i=r2_score(y_test.iloc[:,i], y_pred[:,i])
  print("{} r2 score: {}".format(y_test.columns[i],r2_i))
  r2avg.append(r2_i)
print('Average score: {}'.format(np.average(r2avg)))

best params: {'criterion': 'squared_error', 'max_leaf_nodes': 100, 'min_samples_leaf': 20, 'min_samples_split': 10}
Num Align Hqmt r2 score: 0.9825685540171131
Mode Mt Base Call Read Length Align Hqmt r2 score: 0.5830252750021165
Avg Mt Align Edit Percent Identical r2 score: 0.9884748464783887
Average score: 0.8513562251658727


While not as good as random forest, the decision tree regressor is much faster but really struggles to capture a meaningful prediction for read length, barely getting above 50% or random chance.

Neural Network for multi-output regression

In [69]:
from sklearn.model_selection import RepeatedKFold
from keras.models import Sequential
from keras.layers import Dense
 

In [79]:
# define the model
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(20, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs))
	model.compile(loss='mae', optimizer='adam')
	return model
 
# evaluate a model using repeated k-fold cross-validation
def evaluate_model(X, y):
	results = list()
	n_inputs, n_outputs = X.shape[1], y.shape[1]
	# define evaluation procedure
	cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=1)
	# enumerate folds
	for train_ix, test_ix in cv.split(X):
		# prepare data
		X_train, X_test = X.iloc[train_ix], X.iloc[test_ix]
		y_train, y_test = y.iloc[train_ix], y.iloc[test_ix]
		# define model
		model = get_model(n_inputs, n_outputs)
		# fit model
		model.fit(X_train, y_train, verbose=0, epochs=100)
		# evaluate model on test set
		mae = model.evaluate(X_test, y_test, verbose=0)
		# store result
		print('>%.3f' % mae)
		results.append(mae)
	return results, model
 

In [71]:
#prototype selection using 100 random features
x_proto, idx=rand_prototypes(100, x_train, x_train.columns)

In [80]:
results, model=evaluate_model(x_proto, y_train)

>0.021
>0.016
>0.015
>0.017
>0.018
>0.014
>0.019
>0.019
>0.014
>0.017
>0.017
>0.016
>0.017
>0.019
>0.016
>0.017
>0.018
>0.017
>0.019
>0.017
>0.018
>0.015
>0.015
>0.023
>0.014
>0.015
>0.021
>0.017
>0.019
>0.018


In [82]:

y_pred = model.predict(x_test.iloc[:,idx])
r2avg=[]
for i in range(len(y_test.columns)):
  r2_i=r2_score(y_test.iloc[:,i], y_pred[:,i])
  print("{} r2 score: {}".format(y_test.columns[i],r2_i))
  r2avg.append(r2_i)
print('Average score: {}'.format(np.average(r2avg)))

Num Align Hqmt r2 score: 0.9063653317746675
Mode Mt Base Call Read Length Align Hqmt r2 score: 0.5424466133440273
Avg Mt Align Edit Percent Identical r2 score: 0.9709900902112951
Average score: 0.80660067844333


In this model exploration we used a simple neural network and found relatively poor results compared to the earlier regression models. Notable, read length continues to be hard to predict though prototype selection for random features may be exascerbating the lowered scores. 